In [ ]:
try:
    from kiara_plugin.jupyter import ensure_kiara_plugins
except:
    import sys
    print("Installing 'kiara_plugin.jupyter'...")
    !{sys.executable} -m pip install -q kiara_plugin.jupyter
    from kiara_plugin.jupyter import ensure_kiara_plugins

ensure_kiara_plugins()

In [ ]:
from kiara.api import KiaraAPI

# create the default api instance
api = KiaraAPI.instance()


In [ ]:
# we can use any of the functions of the api instance
# documentation [here](https://dharpa.org/kiara/latest/reference/kiara/interfaces/python_api/__init__/#kiara.interfaces.python_api.KiaraAPI)

api.list_operation_ids()

In [ ]:
# most likely, we'll want to run an operation, let's pick one

op_id = "download.file"

# now, let's check information about the operation, esp. what input(s) it requires
api.retrieve_operation_info(op_id)


In [ ]:
# ok, we need an 'url' value, and an (optional) file_name:

inputs = {
    "url": "https://raw.githubusercontent.com/DHARPA-Project/kiara.examples/main/examples/data/network_analysis/journals/JournalNodes1902.csv",
    "file_name": "JournalNodes1902.csv"
}

# now, run it
results = api.run_job(operation=op_id, inputs=inputs, comment="Download journal nodes from github.")
# ...and print a result preview
results

In [ ]:
# this looks good, we are really only interested in the 'file' output, let's store it in a variable for later:

downloaded_file = results["file"]

# now we want to do something with that file, specifically, convert it into a table
# the operation id for this is:

op_id = "create.table.from.file"

# and let us check information about the operation, esp. its input(s)
api.retrieve_operation_info(op_id)


In [ ]:
# same procedure as before, we assemble the input(s), and run a kiara job

inputs = {
    "file": downloaded_file
}

# now, run it
results = api.run_job(op_id, inputs=inputs, comment="Create table data from nodes csv.")
# ...and print a result preview
results


In [ ]:
# we are really only interested in the 'table' output field, let's get the 'Value' object for it

table_val = results["table"]
table_val  # this displays some general (mostly internal) information and metadata for the value

In [ ]:
# maybe we want to run a sql query against the table, let's find a good operation

api.list_operation_ids("query")

In [ ]:
# 'query.table' looks good, let's see...

api.retrieve_operation_info('query.table')

In [ ]:
# ok, given this information, we only need to provide the 'table' and 'query', and reference the string data in the sql...

inputs = {
    "table": table_val,
    "query": "SELECT Label, City from data where City like 'Berlin'"
}
results = api.run_job("query.table", inputs=inputs, comment="Query label for all 'Berlin' entries.")
results

In [ ]:
# let's get the value of the 'query_result' field...

query_result = results['query_result']

# and have a look at its lineage (even though it's not particularly interesting yet)
query_result.lineage


In [ ]:
# and some (type specific) properties...

query_result.property_values


In [ ]:
# the value is of type 'table', we can access the data as a Python object

query_table_data = query_result.data

# we can check out what (Python) type is used under the hood
type(query_table_data)

In [ ]:
# and if we look up documentation about this type, we can use some of its
# utility methods etc. For this type, the documentation is here: https://dharpa.org/kiara_plugin.tabular/latest/reference/kiara_plugin/tabular/models/table/#kiara_plugin.tabular.models.table.KiaraTable
#
# TODO: implement discoverability for these types of things, so links to documentation can be accessd
# directly via the Python object

# for example, we can get the table as a Pandas dataframe:

df = query_table_data.to_pandas_dataframe()
df
